In [1]:
from gensim.models import KeyedVectors
from tqdm import tqdm
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import re
import numpy as np

In [3]:
spxEM = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300_em.txt'
spxEM_model = KeyedVectors.load_word2vec_format(spxEM, binary=False)
w2v = '/home/manni/embs/en_wiki_w2v_mc100_epoch5_300.txt'
w2v_model = KeyedVectors.load_word2vec_format(w2v, binary=False)
spx = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300.txt'
spx_model = KeyedVectors.load_word2vec_format(spx, binary=False)

In [4]:
models = [w2v_model,spx_model,spxEM_model]
model_names = ['Word2vec','EWEM-MIX','EWEM']

In [ ]:
spxEM = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300_em_w1.txt'
spxEM_model = KeyedVectors.load_word2vec_format(spxEM, binary=False)
w2v = '/home/manni/embs/en_wiki_w2v_mc100_epoch5_300_w1.txt'
w2v_model = KeyedVectors.load_word2vec_format(w2v, binary=False)

In [ ]:
models = [w2v_model,spxEM_model]
model_names = ['Word2vec','EWEM']

# SCWS

In [5]:
scws_file = '/home/manni/data/wordsim/SCWS/ratings.txt'

In [6]:
data = list()
with open(scws_file) as fin:
    lines = fin.readlines()
    for l in lines:
        l = l.lower().split('\t')
        w1 = l[1]
        w2 = l[3]
        c1 = l[5]
        c2 = l[6]
        score = float(l[7])
        data.append([w1,w2,c1,c2,score])

In [7]:
data[6]

['israel',
 'israeli',
 'pathogens are attached to silver and gold nanowires . in the netherlands , the company tno has designed bioaerosol single particle recognition equipment ( biosparq ) . this system would be implemented into the national response plan for bioweapons attacks in the netherlands . researchers at ben gurion university in <b> israel </b> are developing a different device called the biopen , essentially a " lab-in-a-pen " , which can detect known biological agents in under 20 minutes using an adaptation of the elisa , a similar widely employed immunological technique , that in this case incorporates fiber optics . list of',
 '1995 , israeli folk singer miri aloni sang the israeli pop song " shir lashalom " ( song for peace ) . this song , originally written in 1969 and performed extensively at the time by an israeli military performing group , has become one of the anthems of the <b> israeli </b> peace camp . during the arab uprising known as the first intifada , israe

In [25]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def clean_sent(sent):
    '''
    parameters:
        sent: str
    returns:
        tokens: list(str)     
    '''
    return [word for word in sent.split() if word not in stop_words]

def sent_words(sent):
    '''
    parameters:
        sent: str
    returns:
        tokens: list(str) 
    '''
    sent = re.sub(r'[^\w\s]', '', sent)
    left = sent.split('<b>')[0]
    right = sent.split('</b>')[-1]
    sentA = clean_sent(left)[-10:]
    sentB = clean_sent(right)[:10]
    return sentA+sentB

def reducer(wrd,lst,model):
    out = set()
    for word in lst:
        top = [tup[0] for tup in model.most_similar(word,topn=100)]
        out.update(set(top)&set(lst))
    out.add(wrd)
    if wrd+'#E' in model.vocab:
        out.add(wrd+'#E')
    return out

def avgsim(w1,w2,c1,c2,score,model,reduce=False,verbose=False):
    '''
    Computes average similarity score.

    parameters:
        w1 : str
        w2 : str
        c1 : str
        c2 : str
        model : gensim.keyedvectors
    returns:
        score : float
    '''
    if w1 not in model.vocab or w2 not in model.vocab:
        return None
    a = [w1] + sent_words(c1)
    _a = [word+'#E' for word in a] 
    a+=_a
    b = [w2] + sent_words(c2)
    _b = [word+'#E' for word in b] 
    b+=_b
    a = [word for word in a if word in model.vocab]
    b = [word for word in b if word in model.vocab]
    if reduce:
        a = reducer(w1,a,model)
        b = reducer(w2,b,model)    
    if verbose:
        print(a)
        print(b)
    div = len(a)*len(b)
    sims = 0
    for i in a:
        for j in b:
            sim = model.similarity(i,j)
            if verbose:
                print(i,j,sim)
            sims+= sim
    return (sims/div,score)

def _avgvec(lst,model):
    vec = np.zeros(model.vector_size)
    for word in lst:
        if word in model:
            vec = np.add(vec,model.get_vector(word))
    return vec  
    
def avgvec(w1,w2,c1,c2,score,model,reduce=False,verbose=False):
    if w1 not in model.vocab or w2 not in model.vocab:
        return None
    a = [w1] + sent_words(c1)
    _a = [word+'#E' for word in a] 
    a+=_a
    b = [w2] + sent_words(c2)
    _b = [word+'#E' for word in b] 
    b+=_b
    a = [word for word in a if word in model.vocab]
    b = [word for word in b if word in model.vocab]
    if reduce:
        a = reducer(w1,a,model)
        b = reducer(w2,b,model) 
    if verbose:
        print(a)
        print(b) 
    v1 = _avgvec(a,model)
    v2 = _avgvec(b,model)
    return (model.cosine_similarities(v1,[v2])[0],score)

In [21]:
def get_corr(model,method='asim',reduce=False):
    sims = list()
    scores = list()
    for dat in tqdm(data,position=0):
        if method == 'asim':
            sim = avgsim(dat[0],dat[1],dat[2],dat[3],dat[4],model,reduce=reduce)
        if method == 'avg':
            sim = avgvec(dat[0],dat[1],dat[2],dat[3],dat[4],model,reduce=reduce)
        if sim:
            sims.append(sim[0])
            scores.append(sim[1])
    corr, _ = pearsonr(sims, scores)
    print('%.2f &' % (corr*100),end=' ')
    corr, _ = spearmanr(sims, scores)
    print('%.2f' % (corr*100))   

In [22]:
get_corr(spxEM_model)
get_corr(spx_model)
get_corr(w2v_model)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2003/2003 [00:00<00:00, 8461.01it/s]


60.69 & 61.19


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2003/2003 [00:00<00:00, 29723.68it/s]


63.45 & 65.14


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2003/2003 [00:00<00:00, 26045.20it/s]

62.92 & 63.83


In [23]:
get_corr(spxEM_model,method='avg')
get_corr(spx_model,method='avg')
get_corr(w2v_model,method='avg')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2003/2003 [00:00<00:00, 12052.46it/s]


61.88 & 61.99


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2003/2003 [00:00<00:00, 16184.09it/s]


63.45 & 65.13


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2003/2003 [00:00<00:00, 16474.25it/s]

62.92 & 63.78


In [ ]:
from multiprocessing import Pool

def get_corr_parallel(model,method='asim',reduce=False):
    if __name__=="__main__":
        pool = Pool(processes=40)
        sims = list()
        scores = list()
        for dat in tqdm(data,position=0):
            if method == 'asim':
                sim = pool.apply_async(func=avgsim, args=(dat[0],dat[1],dat[2],dat[3],dat[4],model,reduce)) 
            if method == 'avg':
                sim = pool.apply_async(func=avgvec, args=(dat[0],dat[1],dat[2],dat[3],dat[4],model,reduce)) 
            sim = sim.get()
            if sim:
                sims.append(sim[0])
                scores.append(sim[1])
        pool.close()
        pool.join()
        corr, _ = pearsonr(sims, scores)
        print('Pearsons correlation: %.2f' % (corr*100))
        corr, _ = spearmanr(sims, scores)
        print('Spearmans correlation: %.2f' % (corr*100)) 

In [ ]:
get_corr_parallel(spxEM_model,reduce=True)
get_corr_parallel(spx_model,reduce=True)
get_corr_parallel(w2v_model,reduce=True)

In [26]:
get_corr(spxEM_model,reduce=True)
get_corr(spx_model,reduce=True)
get_corr(w2v_model,reduce=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2003/2003 [41:46<00:00,  1.25s/it]


29.67 & 30.12


 56%|███████████████████████████████████████████████████████████████████████████████████████▎                                                                     | 1114/2003 [10:01<07:55,  1.87it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
get_corr(spxEM_model,method='avg',reduce=True)
get_corr(spx_model,method='avg',reduce=True)
get_corr(w2v_model,method='avg',reduce=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2003/2003 [40:47<00:00,  1.22s/it]


30.83 & 29.67


 51%|███████████████████████████████████████████████████████████████████████████████▋                                                                             | 1016/2003 [08:53<08:06,  2.03it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(w2v_model.similarity('brazil','nut'))
print(spx_model.similarity('brazil','nut'))
print(spxEM_model.similarity('brazil','nut'))
print(spxEM_model.similarity('brazil#E','nut#E'))

In [ ]:
print(spx_model.similarity('new','york'))
print(w2v_model.similarity('new','york'))
print(spxEM_model.similarity('new','york'))
print(spxEM_model.similarity('new#E','york#E'))

In [ ]:
avgsim(data[0][0],data[0][1],data[0][2],data[0][3],w2v_model,verbose=True)

In [ ]:
x = ['brazil', 'bahia', 'instance', 'never', 'gap', 'income', 'blacks', 'nonwhites', 'relatively', 'brazil#E', 'bahia#E', 'instance#E', 'never#E', 'gap#E', 'income#E', 'blacks#E', 'relatively#E']
y = ['nut', 'player', 'turns', 'change', 'string', 'tension', 'neck', 'bridge', 'pickups', 'features', 'found', 'nut#E', 'player#E', 'turns#E', 'change#E', 'string#E', 'tension#E', 'neck#E', 'bridge#E', 'pickups#E', 'features#E', 'found#E']

In [ ]:
x = ['brazil', 'population', 'black', 'politicians', 'city', 'salvador', 'bahia', 'instance', 'never', 'gap', 'income', 'blacks', 'nonwhites', 'relatively', 'small', 'compared', 'large', 'gap', 'whites', 'brazil#E', 'population#E', 'black#E', 'politicians#E', 'city#E', 'salvador#E', 'bahia#E', 'instance#E', 'never#E', 'gap#E', 'income#E', 'blacks#E', 'relatively#E', 'small#E', 'compared#E', 'large#E', 'gap#E', 'whites#E']
y = ['nut', 'machine', 'heads', 'worm', 'gears', 'player', 'turns', 'change', 'string', 'tension', 'neck', 'bridge', 'pickups', 'features', 'found', 'almost', 'every', 'guitar', 'photo', 'shows', 'nut#E', 'machine#E', 'heads#E', 'worm#E', 'gears#E', 'player#E', 'turns#E', 'change#E', 'string#E', 'tension#E', 'neck#E', 'bridge#E', 'pickups#E', 'features#E', 'found#E', 'almost#E', 'every#E', 'guitar#E', 'photo#E', 'shows#E']

In [ ]:
x = ['israel', 'case', 'incorporates', 'fiber', 'optics', 'list', 'pathogens', 'attached', 'silver', 'gold', 'nanowires', 'israel#E', 'case#E', 'fiber#E', 'optics#E', 'list#E', 'pathogens#E', 'attached#E', 'silver#E', 'gold#E']
y = ['israeli', 'policy', 'territories', 'song', 'banned', 'radio', 'israeli', 'folk', 'singer', 'miri', 'israeli#E', 'policy#E', 'territories#E', 'song#E', 'banned#E', 'radio#E', 'israeli#E', 'folk#E', 'singer#E', 'miri#E']

In [ ]:
x = ['israel', 'similar', 'widely', 'employed', 'immunological', 'technique', 'case', 'incorporates', 'fiber', 'optics', 'list', 'pathogens', 'attached', 'silver', 'gold', 'nanowires', 'netherlands', 'company', 'tno', 'designed', 'israel#E', 'similar#E', 'widely#E', 'employed#E', 'immunological#E', 'technique#E', 'case#E', 'fiber#E', 'optics#E', 'list#E', 'pathogens#E', 'attached#E', 'silver#E', 'gold#E', 'netherlands#E', 'company#E', 'tno#E', 'designed#E']
y = ['israeli', 'written', 'shalom', 'hanoch', 'protest', 'israeli', 'policy', 'territories', 'song', 'banned', 'radio', 'israeli', 'folk', 'singer', 'miri', 'aloni', 'sang', 'israeli', 'pop', 'song', 'israeli#E', 'written#E', 'shalom#E', 'hanoch#E', 'protest#E', 'israeli#E', 'policy#E', 'territories#E', 'song#E', 'banned#E', 'radio#E', 'israeli#E', 'folk#E', 'singer#E', 'miri#E', 'aloni#E', 'sang#E', 'israeli#E', 'pop#E', 'song#E']

In [ ]:
v1 = get_avgvec(x,w2v_model)
v2 = get_avgvec(y,w2v_model)

w2v_model.cosine_similarities(v1,[v2])

In [ ]:
v1 = get_avgvec(x,spx_model)
v2 = get_avgvec(y,spx_model)

spx_model.cosine_similarities(v1,[v2])

In [ ]:
v1 = get_avgvec(x,spxEM_model)
v2 = get_avgvec(y,spxEM_model)

spxEM_model.cosine_similarities(v1,[v2])

In [ ]:
avgsim(data[0][0],data[0][1],data[0][2],data[0][3],spx_model,verbose=True)

In [ ]:
avgsim(data[0][0],data[0][1],data[0][2],data[0][3],spxEM_model,verbose=True)

In [ ]:
avgsim(data[6][0],data[6][1],data[6][2],data[6][3],spxEM_model,verbose=True)

In [ ]:
for w in model.vocab:
    if '#' in w:
        print(w)
        break

In [ ]:
import wiki as w
import sys
sys.path.append("../../imports/")
import saver as sv

In [ ]:
sentences = sv.load("wiki_sentences_spx")

In [ ]:
for sent in sentences:
    print(sent[:105])
    break

In [ ]:
print(data[0][0])
print(sent_words(data[0][2]))
print(data[0][1])
print(sent_words(data[0][3]))

In [ ]:
spxEM_model.similarity('brazil','nut')

In [ ]:
spxEM_model.similarity('brazil#E','nut#E')

In [ ]:
w2v_model.similarity('brazil','nut')

In [ ]:
sent_words(c1)

# non-context similarities

In [ ]:
ws353A = '/home/manni/data/wordsim/EN-WS353.txt'
ws353R = '/home/manni/data/wordsim/EN-WSR353.txt'
ws353S = '/home/manni/data/wordsim/EN-WSS353.txt'
rw = '/home/manni/data/wordsim/rw.txt'
sim999 = '/home/manni/data/wordsim/EN-SIM999.txt'
turk = '/home/manni/data/wordsim/EN_TRUK.txt'
mturk = '/home/manni/data/wordsim/MTURK-771.csv'
rg = '/home/manni/data/wordsim/EN-RG-65.txt'
men = '/home/manni/data/wordsim/EN-MEN-LEM.txt'

In [ ]:
with open(sim999) as fin:
    lines = fin.readlines()
    with open(sim999+'_new','w') as fout:
        for line in lines:
            line = line.split('\t')
            try:
                float(line[3])
                fout.write("{} {} {} \n".format(line[0],line[1],line[3])) 
            except:
                continue   
sim999 = sim999+'_new'

In [ ]:
with open(mturk) as fin:
    lines = fin.readlines()
    with open(mturk+'_new','w') as fout:
        for line in lines:
            line = line.split(',')
            try:
                fout.write("{} {} {} \n".format(line[0],line[1],line[2])) 
            except:
                continue   
mturk = mturk+'_new'

In [ ]:
with open(men) as fin:
    lines = fin.readlines()
    with open(men+'_new','w') as fout:
        for line in lines:
            line = line.split()
            try:
                fout.write("{} {} {} \n".format(line[0].split('-')[0],line[1].split('-')[0],line[2])) 
            except:
                continue   
men = men+'_new'

In [ ]:
# debug
with open(mturk) as fin:
    lines = fin.readlines()
    for line in lines:
        line= line.split()
        print(line)
        continue
        try:
            if line[0] in model.vocab and line[1] in model.vocab:
                continue
        except:
            print(line)

In [ ]:
datasets = [ws353A,ws353R,ws353S,rw,sim999,turk,mturk,rg,men]

In [ ]:
def avg_sim(w1,w2,model):
    s1 = [w1,w1+'#E']
    s2 = [w2,w2+'#E']
    a = [w for w in s1 if w in model.vocab]
    b = [w for w in s2 if w in model.vocab]
    div = len(a)*len(b)
    sims = 0
    for i in a:
        for j in b:
            sims+=model.similarity(i,j)
    return sims/div

In [ ]:

for ds in datasets:
    name = ds.split('/')[-1].split('.')[0]
    print(r'\begin{subsection}{'+name+r'}')
    print(r'\begin{table}[!h]')
    print(r'\begin{tabular}{|l|c|c|}')
    print('\hline')
    print("Model & Pearsons & Spearmans"+r"\\")
    print('\hline')
    with open(ds) as fin:
        lines = fin.readlines()
        for i, model in enumerate(models):
            print(model_names[i],end=' & ')
            sims = list()
            scores = list()
            for line in lines:
                line = line.split()
                if not line:
                    continue
                if line[0] in model.vocab and line[1] in model.vocab:
                    sim = avg_sim(line[0],line[1],model) 
                else:
                    continue
                if sim:
                    score = float(line[2])
                    sims.append(sim)
                    scores.append(score)
            corr, _ = pearsonr(sims, scores)
            print('%.2f' % (corr*100),end=' & ')
            corr, _ = spearmanr(sims, scores)
            print('%.2f' % (corr*100),end=r'\\') 
            print()
    print('\hline')
    print('\end{tabular}')
    print('\end{table}')
    print(r'\end{subsection}')
    print()